# Introduction

The DREAM R package provides a comprehensive set of functionalities to explore and predict drug combinations based on gene expression and perturbation data. 
In this vignette, we outline the main steps of the analysis and explain how to utilize the package.

### Analysis Workflow

The analysis workflow consists of the following steps:
  1. Building a Disease Network: The first step involves constructing a disease network based on differential gene expression profiles. 
  This network captures the interactions between genes and serves as the foundation for subsequent analyses.

  2. Identifying Druggable Target Genes: Next, we identify the druggable target genes within the disease network.
     By pinpointing specific genes that can be targeted by drugs, we gain valuable insights into potential therapeutic opportunities.

  3. Building Drug Similarity Matrices: To assess the similarity between drugs, we construct multiple drug distance metrics based on molecular and chemical aspects.
     These distances enable us to evaluate the relationships and potential combinations among different drugs.

  4. Predicting Drug Combinations: Leveraging the disease network, druggable target genes, and drug distances, we utilize a multi-objective genetic algorithm to predict and prioritize potential drug combinations.
     This step is crucial in identifying drug interactions and optimizing therapeutic interventions.

### Required Inputs

To perform the analysis, the following inputs are required:

  1. Gene Expression Experiments: At least two sets of gene expression experiments representing different experimental conditions are required.
     These experiments should capture the transcriptional responses under various conditions or treatments.

  2. Transcriptional Perturbation Profiles: Perturbation profiles derived from drug treatments are essential for identifying druggable targets and predicting drug combinations.
  These profiles provide valuable information on how genes are affected by specific drugs.

### Warning: Execution time
Please note that some of the code chunks in this analysis may take a significant amount of time to execute, especially when dealing with large datasets or performing computationally intensive tasks.

To minimize waiting time, we provide intermediate output files for certain steps [here](https://github.com/fhaive/dream/releases/download/public/intermediate.tar.gz).
These files contain pre-computed results that can be loaded directly, without the need to re-run time-consuming code chunks.

### Python setup

Some steps of the analysis require python to run, so the path to the interpreter must be provided.
The dream package comes with both python and R environments with all the necessary dependencies listed in the `environment.yml` file.
To get the path of the python interpreter it is sufficient to run these commands in a bash shell:
```
conda run -n dream which python
```
Copy and paste this path in the chunk below as the value of the variable `python_bin`.

In [ ]:
python_bin = "/opt/mambaforge/bin/python"
library(DREAM)

# setup data paths, the root directory can be overridden by changing the following instruction
rootdir = "/workspace"
data_path = file.path(rootdir, "data")
intermediate_path = file.path(rootdir, "intermediate")

dir.create(data_path, showWarnings=FALSE)
# this folder containes intermediate files, they can either be generated by the notebook, or they can be downloaded from [...] to save computation time
dir.create(intermediate_path, showWarnings=FALSE)
drug_expr_path = file.path(intermediate_path, "drug_expr")

# Generating Co-expression Disease Network for Atopic Dermatitis
In this step, we will generate a co-expression disease network for Atopic Dermatitis (AD) based on informative genes identified as differentially expressed genes (DEGs).

## Data Retrieval
We retrieve the gene expression data for both lesional and non-lesional AD conditions. 
We will use the `download.file` function to download the data files from Zenodo. 
The URLs for the data files are:

  Lesional AD data: [GE_Mic_AD_Pamr_MAARS.txt](https://zenodo.org/record/7962332/files/GE_Mic_AD_Pamr_MAARS.txt?download=1)
  Non-lesional AD data: [GE_Mic_AD_Pamr_nl_MAARS.txt](https://zenodo.org/record/7962332/files/GE_Mic_AD_Pamr_nl_MAARS.txt?download=1)

The code for downloading the data files is as follows:

In [ ]:
options(timeout=3600)
ad_gex_url <- "https://zenodo.org/record/7962332/files/GE_Mic_AD_Pamr_MAARS.txt?download=1"
ad_gex_nl_url <- "https://zenodo.org/record/7962332/files/GE_Mic_AD_Pamr_nl_MAARS.txt?download=1"
download.file(ad_gex_url, file.path(data_path, "GE_Mic_AD_Pamr_MAARS.txt"), method="wget", )
download.file(ad_gex_nl_url, file.path(data_path, "GE_Mic_AD_Pamr_nl_MAARS.txt"), method="wget", )

## Data Preprocessing
We define a function to process the downloaded gene expression data. 
This function reads the data file, retrieves the gene symbol information using Ensembl gene IDs, removes rows with missing symbols, and returns the processed expression data.

In [ ]:
process_gene_expression_data <- function(data_file, ensembl_dataset) {
  expression_data <- read.table(data_file, header = TRUE, sep = "\t", quote = "")
  
  conversion_table <- biomaRt::getBM(
    attributes = c("ensembl_gene_id", "hgnc_symbol"),
    filters = "ensembl_gene_id",
    values = rownames(expression_data),
    mart = ensembl_dataset
  )
  
  conversion_table$hgnc_symbol[which(conversion_table$hgnc_symbol == "")] <- NA
  
  expression_data$symbols <- conversion_table$hgnc_symbol[match(rownames(expression_data), conversion_table$ensembl_gene_id)]
  expression_data <- na.omit(expression_data)
  rownames(expression_data) <- expression_data$symbols
  expression_data$symbols <- NULL
  
  return(expression_data)
}

# Note: if these commands return a biomaRt server error try to rerun them
ensembl <- biomaRt::useMart("ensembl", dataset = "hsapiens_gene_ensembl")

ad_pamr <- process_gene_expression_data(file.path(data_path, "GE_Mic_AD_Pamr_MAARS.txt"), ensembl)
ad_pamr_nl <- process_gene_expression_data(file.path(data_path, "GE_Mic_AD_Pamr_nl_MAARS.txt"), ensembl)

Next, we identify the common genes present in both lesional and non-lesional AD data. 

In [ ]:
common_genes <- intersect(rownames(ad_pamr), rownames(ad_pamr_nl))
ad_pamr <- ad_pamr[which(rownames(ad_pamr) %in% common_genes),]
ad_pamr_nl <- ad_pamr_nl[which(rownames(ad_pamr_nl) %in% common_genes),]

Finally, we select the informative genes (differentially expressed genes).
We will use the `get_informative_genes` function, passing the combined data frame and the grouping information as parameters. 
In this case, the grouping information represents whether a sample is from the lesional AD group or the non-lesional AD group. 
The `test` parameter specifies the statistical test to use for gene selection, and the `percentile` parameter determines the threshold for selecting the top informative genes.

In [ ]:
# this computation can be skipped by loading this intermediate file
# inputinform <- readRDS(file.path(intermediate_path, "inputinform.rds"))
all_samples_filtered <- cbind(ad_pamr, ad_pamr_nl)
inputinform <- get_informative_genes(expr_mat = all_samples_filtered, group = as.factor(c(rep("AD", ncol(ad_pamr)), rep("N", ncol(ad_pamr_nl)))), test = "wilcoxon + var.test", percentile = 10)

## Consensus Matrix Generation
To create the co-expression network, we will use the `get_ranked_consensus_matrix` function. 
This function takes the gene expression table (`inputinform[[1]]`) containing the informative genes as input. 
The `gx_table` parameter represents the gene expression data for the DEGs.

Several parameters can be specified to configure the co-expression network generation, such as the methods for network inference (`iMethods`), the correlation/association estimation method (`iEst`), and the discretization method (`iDisc`).
We use all the possible combinations of values in order to obtain a more robust consensus co-expression network.

In [ ]:
# Note: Warning - This code chunk takes a significant amount of time to run.
generatematrices=get_ranked_consensus_matrix(
    gx_table=inputinform[[1]], 
    iMethods=c("clr","aracne","mrnet"),
    iEst=c("pearson","spearman","kendall","mi.empirical","mi.mm","mi.shrink","mi.sg"),
    iDisc=c("none","equalfreq","equalwidth","globalequalwidth"),
    ncores=32,
    debug_output=TRUE,
    updateProgress=TRUE
)

## Edge Selection
Next, we parse the edge rank matrix obtained from the consensus matrix and perform edge selection based on different strategies. 
The `parse_edge_rank_matrix` function is used for this purpose.

The function takes the edge rank matrix (`edge_rank_matrix`) as input and specifies the edge selection strategy (`edge_selection_strategy`).
Additionally, the `mat_weights` parameter determines the weight attribute used for edge selection, and `topN` specifies the number of top-ranked edges to select.

In [ ]:
rankMat.parsed <- parse_edge_rank_matrix(
    edge_rank_matrix=generatematrices,
    edge_selection_strategy="default",
    mat_weights="rank",
    topN=10,
    debug_output=TRUE,
    updateProgress=TRUE
)

We create an iGraph object from the symmetrical adjacency matrix. This object annotated with centrality attributes will be used in following steps of the analysis.
Finally, we convert the iGraph object into a data frame format (`disease_network`) that represents the edges in the network.
Each row of the data frame contains two genes representing an edge in the network.

In [ ]:
# this computation can be skipped by loading this intermediate file
# conGraph <- readRDS(file.path(intermediate_path, "conGraph.rds"))
conGraph <- get_iGraph(rankMat.parsed$bin_mat)
disease_network <- data.frame(as_edgelist(as.undirected(conGraph, mode="collapse")))
colnames(disease_network) <- c("gene1", "gene2")

## Ranking Genes in the Disease Network
We rank the genes in the disease network based on their differential expression. 
We use the `get_graph_named_ranklist` function, which takes an iGraph object (`conGraph`) and a list of ranked vectors as input.
The `get_graph_named_ranklist` function assigns the ranked vectors as vertex attributes in the graph and calculates a ranked gene list using specified network attributes. 
The resulting ranked gene list is returned as a named list.
In this case, the ranked vectors used for ranking the genes are `inputinform[["var_score"]]` and `inputinform[["wilcox_score"]]`. 
These vectors represent the scores of rankings of genes based on their variability and significance in the differential expression analysis.
We convert the ranked gene list into a data frame format. 
Each row of the data frame contains a gene name and its corresponding rank.

In [ ]:
graph_rank <- get_graph_named_ranklist(
    Graph=conGraph,
    ranked_vectors=list(inputinform[["var_score"]],
    inputinform[["wilcox_score"]])
)
rank <- data.frame(gene=names(graph_rank), rank=graph_rank)

# Druggability evaluation of the disease under investigation and computation of drug properties.

## Downloading OpenTargets JSON File
First, we download the OpenTargets JSON file from the specified URL (`opentargets_url`) using the `download.file` function. 
The downloaded file will be saved as "./data/19.02_evidence_data.json.gz". 
This file is in a compressed format (`Gzip`).

In [ ]:
# Note: Warning - This code chunk takes a significant amount of time to run.
opentargets_url <- "http://ftp.ebi.ac.uk/pub/databases/opentargets/platform/19.02/output/19.02_evidence_data.json.gz"
download.file(opentargets_url, file.path(data_path, "19.02_evidence_data.json.gz"), method="wget", )

## Parsing OpenTargets JSON
Next, we parse the downloaded OpenTargets JSON file using a custom convenience function called `parse_opentargets_json`.
This function takes the JSON file (in Gzip format) as input and returns a parsed data frame with specific fields extracted.

In [ ]:
# Note: Warning - This code chunk takes a significant amount of time to run.
# the intermediate result final_opentargets_parsed can be loaded with
# final_opentargets_parsed <- readRDS(file.path(intermediate_path, "final_opentargets_parsed.rds"))
final_opentargets_parsed = parse_opentargets_json(gzfile(file.path(data_path, "19.02_evidence_data.json.gz")))

## Mapping OpenTargets Data on the Disease Network
To identify druggable targets on the disease network, we map the OpenTargets data onto the disease network graph (`conGraph`).
The OpenTargets data, which is stored in the `final_opentargets_parsed` data frame in the previous step, will be used for this mapping.

The `map_opentargets_on_graph` function is designed to map OpenTargets data onto a graph.
It takes the disease network graph and the parsed OpenTargets data as inputs.

In [ ]:
map_drugs_on_graph <- map_opentargets_on_graph(Graph = conGraph, parsed_opentargets = final_opentargets_parsed)
map_drugs_on_graph <- unique(map_drugs_on_graph)

## Downloading OpenTargets JSON File
We download the LINCS L1000 files from the list of URLs (`lincs_urls`) using the `download.file` function. 
The downloaded file will be saved as "./data/GSE92742/*". 

In [ ]:
lincs_urls <- list(
  "GSE92742_Broad_LINCS_pert_info.txt"="https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE92742&format=file&file=GSE92742%5FBroad%5FLINCS%5Fpert%5Finfo%2Etxt%2Egz",
  "GSE92742_Broad_LINCS_sig_info.txt"="https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE92742&format=file&file=GSE92742%5FBroad%5FLINCS%5Fsig%5Finfo%2Etxt%2Egz",
  "GSE92742_Broad_LINCS_gene_info_delta_landmark.txt"="https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE92742&format=file&file=GSE92742%5FBroad%5FLINCS%5Fgene%5Finfo%5Fdelta%5Flandmark%2Etxt%2Egz",
  "GSE92742_Broad_LINCS_Level3_INF_mlr12k_n1319138x12328.gctx"="https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE92742&format=file&file=GSE92742%5FBroad%5FLINCS%5FLevel3%5FINF%5Fmlr12k%5Fn1319138x12328%2Egctx%2Egz"
)
l1000_path = file.path(data_path, "L1000")
dir.create(l1000_path, showWarnings = FALSE)
for(filename in names(lincs_urls)){
  download.file(lincs_urls[[filename]], file.path(l1000_path, filename))
}

## Importing L1000 drug perturbations
In this step we identify relevant drugs from the LINCS L1000 dataset.
We read the L1000 perturbation information file and select those drugs that have known targets mapped on the co-expression disease network.

In [ ]:
L1000_GSE92742_pert_info <- read.delim(file.path(l1000_path, "GSE92742_Broad_LINCS_pert_info.txt"), header=TRUE, quote = "")
toRemove <- c(1,3,4,5,6,8)
L1000_GSE92742_pert_info <- L1000_GSE92742_pert_info[, -toRemove]
L1000_GSE92742_pert_info$pert_iname <- sapply(L1000_GSE92742_pert_info$pert_iname, tolower)

map_drugs_on_graph$dat.drug.molecule_name <- gsub(map_drugs_on_graph$dat.drug.molecule_name, pattern = ".", replacement = "-", fixed = TRUE)
filter_by_L1000_drugs <- map_drugs_on_graph[which(map_drugs_on_graph$dat.drug.molecule_name %in% L1000_GSE92742_pert_info$pert_iname), ]
filter_by_L1000_drugs$canonical_smiles <- NA
filter_by_L1000_drugs$canonical_smiles <- L1000_GSE92742_pert_info$canonical_smiles[match(filter_by_L1000_drugs$dat.drug.molecule_name, L1000_GSE92742_pert_info$pert_iname)]
filter_by_L1000_drugs <- unique(filter_by_L1000_drugs)

## Selecting Cell Type and Filtering L1000 Metadata
We filter the signature metadata to include only entries where the experimental conditions match the ones of interest: `cell_id` is FIBRNPC, `pert_idose` is "10 µM", `pert_itime` is "24 h", and `pert_iname` is present in `filter_by_L1000_drugs$dat.drug.molecule_name`.
We do a similar filtering for the control samples, with the exception that in this case the type of perturbation is set to "DMSO".

In [ ]:
cell_type <- "FIBRNPC"

col_meta_path_2 <- file.path(l1000_path, "GSE92742_Broad_LINCS_sig_info.txt")
col_meta_2 <- read.delim(col_meta_path_2, sep = "\t", stringsAsFactors = F)
col_path_2 <- file.path(l1000_path, "GSE92742_Broad_LINCS_Level3_INF_mlr12k_n1319138x12328.gctx")

col_meta_filtered_2 <- col_meta_2 %>%
  dplyr::filter(cell_id==cell_type, pert_idose=="10 µM", pert_itime=="24 h") %>%
  dplyr::filter(pert_iname %in% filter_by_L1000_drugs$dat.drug.molecule_name) 

col_meta_filtered_2_CTRL <- col_meta_2 %>%
  dplyr::filter(cell_id==cell_type, pert_itime=="24 h", pert_iname=="DMSO")

col_meta_filtered_2[, c(6,7,9,10)] <- NULL
col_meta_filtered_2_CTRL[, c(6,7,9,10)] <- NULL

col_meta_filtered_2 <- tidyr::separate_rows(col_meta_filtered_2, distil_id, sep = "\\|")
col_meta_filtered_2 <- unique(col_meta_filtered_2)
rownames(col_meta_filtered_2) <- 1:nrow(col_meta_filtered_2)

col_meta_filtered_2_CTRL <- tidyr::separate_rows(col_meta_filtered_2_CTRL, distil_id, sep = "\\|")
rownames(col_meta_filtered_2_CTRL) <- 1:nrow(col_meta_filtered_2_CTRL)
col_meta_filtered_2_CTRL <- col_meta_filtered_2_CTRL[-which(col_meta_filtered_2_CTRL$pert_idose!="-666"),]

## Loading L1000 expression data
We are ready to extract the expression data for our samples of interest from the Level 3 of the L1000 dataset.

In [ ]:
GSE92742_ds <- parse.gctx(col_path_2, cid = col_meta_filtered_2$distil_id)
GSE92742_CTRL <- parse.gctx(col_path_2, cid=unique(col_meta_filtered_2_CTRL$distil_id))

We read the file "GSE92742_Broad_LINCS_gene_info_delta_landmark.txt" to obtain information about landmark genes and store the gene IDs in the `landmark_genes` variable as character strings.

In [ ]:
landmark_genes <- read.delim(file.path(l1000_path, "GSE92742_Broad_LINCS_gene_info_delta_landmark.txt"), header=TRUE, quote="")
landmark_genes <- as.character(landmark_genes$pr_gene_id)

Finally, we create an expression matrix for each drug using the `build_drug_expression_matrices` function. 
This function takes the L1000 data, L1000 control data, L1000 metadata, L1000 control metadata, a destination path, and the landmark genes as input.
The resulting expression matrices, one for each drug, are saved in the "./Expression_matrices_L1000" directory in tab-delimited format.

In [ ]:
create_L1000_exprmat <- build_drug_expression_matrices(L1000_data = GSE92742_ds, L1000_data_CTRL = GSE92742_CTRL, L1000_metadata = col_meta_filtered_2, L1000_metadata_CTRL = col_meta_filtered_2_CTRL, include_controls = TRUE, dest_path = drug_expr_path, inputgenes = landmark_genes)

## Random control samples selection

In this step, we will balance the number of samples between treatment and control groups when creating the expression matrices for each drug.
The reason behind this approach is to avoid an imbalance in the number of columns, where the treatment matrices would have much fewer columns than the control matrices.
Such an imbalance could lead to biased network analysis and produce highly similar networks for all drugs.

To address this issue, we will randomly select control samples to match the number of samples in the treatment group for each drug.
This balancing step is essential to prevent the dominance of control samples and facilitate meaningful comparisons between treatment and control conditions.

We iterate over the files in the "Expression_matrices_L1000/" directory and for each expression matrix, we will select a random set of control samples equal in number to the treatment samples.
The modified matrix is then saved back to the same file location.

In [ ]:
myfiles <- list.files(drug_expr_path, pattern = "Expression_matrix")

for (i in 1:length(myfiles)){
  random_CTRL_matrix <- data.frame()
  file <- read.table(file.path(drug_expr_path, myfiles[i]), header = TRUE, sep = "\t", quote = "", row.names = 1)
  CTRL <- grep(colnames(file), pattern = "CTRL", fixed = TRUE)
  random_CTRL_matrix <- file[, -CTRL]
  random_CTRL_matrix <- cbind(random_CTRL_matrix, file[, sample(x = CTRL, size = ncol(random_CTRL_matrix))])
  write.table(random_CTRL_matrix, file = file.path(drug_expr_path, myfiles[i]), quote = FALSE, sep = "\t", row.names=TRUE, col=NA)
}

## Creating Drug Co-Expression Matrices

In this step, we will create drug co-expression matrices using the perturbation profiles of the drugs in the L1000 dataset, similar to how we constructed the disease network.

In [ ]:
myfiles <- list.files(drug_expr_path, pattern = "Expression_matrix")

inputinform_drugs <- sapply(strsplit(myfiles, split = "_", fixed = TRUE), "[", 3)
inputinform_drugs <- sapply(strsplit(inputinform_drugs, split = ".", fixed = TRUE), "[", 1)

for(i in 1:length(myfiles)){
  file <- read.table(file.path(drug_expr_path, myfiles[i]), header = TRUE, sep = "\t", quote = "", row.names = 1)
  
  generatematrices=get_ranked_consensus_matrix(gx_table = file, iMethods = c("clr","aracne","mrnet"),
                                               iEst = c("pearson","spearman","kendall","mi.empirical","mi.mm","mi.shrink","mi.sg"),
                                               iDisc=c("none","equalfreq","equalwidth","globalequalwidth"), ncores = 20, debug_output = TRUE, updateProgress = TRUE)
  
  
  #Parse ranked matrix and get bin_mat and edge_rank
  # Get edge rank list and binary inference matrix from edge rank matrix computed by get_ranked_consensus_matrix().
  # parse_edge_rank_matrix parses the edge rank matrix created by using the internal function get_ranked_consensus_matrix_matrix() to get a ranked edge list and a binary matrix.
  
  rankMat.parsed=parse_edge_rank_matrix(edge_rank_matrix = generatematrices, edge_selection_strategy = "default",
                                        mat_weights = "rank", topN = 10, debug_output = TRUE, updateProgress = TRUE)
  
  drug_conGraph <- get_iGraph(rankMat.parsed$bin_mat)
  saveRDS(drug_conGraph, file=file.path(drug_expr_path, paste0("conGraph_", inputinform_drugs[i], ".rds")))
}

## Computing Pairwise Distances among Drug Co-Expression Networks

Here, we utilize the `nettools` package to compute the distances between pairs of drug co-expression networks.
The `netdist()` function takes the drug co-expression networks stored in the `infer_L1000_coexpr_networks` list as input.
We specify the distance metric as "HIM", which stands for Hamming and Ipsen-Mikhailov, a measure for quantifying network differences.

The computed distances are stored in the `ddist` object.
We assign meaningful row and column names to the distance matrix, matching them with the drug names converted to lowercase.

Finally, we construct a network that represents the distances between drugs based on their perturbational differences in gene expression profiles.
This network offers a comprehensive view of the relationships and similarities among the drugs.
The resulting `moa` object represents the drug distance network as a dataframe, where each row and column corresponds to a drug, and the cell values indicate the distance between them.

In [ ]:
myfiles <- list.files(path = drug_expr_path, pattern = "conGraph")

# load drug co-expression netoworks based on the perturbational profiles of the 1000 landmark genes of L1000
infer_L1000_coexpr_networks <- list()
for (i in 1:length(myfiles)){
  file <- readRDS(file.path(drug_expr_path, myfiles[i]))
  infer_L1000_coexpr_networks[[i]] <- file
}
names(infer_L1000_coexpr_networks) <- sapply(strsplit(myfiles, ".", fixed=TRUE), "[", 1)
names(infer_L1000_coexpr_networks) <- sapply(strsplit(names(infer_L1000_coexpr_networks), "_", fixed=TRUE), "[", 2)

# compute the pairwise distances among the drug co-expression networks
ddist <- nettools::netdist(infer_L1000_coexpr_networks, d="HIM", n.cores=4)
mat_dist_mat <- ddist$HIM
rownames(mat_dist_mat) <- colnames(mat_dist_mat) <- tolower(names(infer_L1000_coexpr_networks))
# network of drug distances based on perturbational differences
moa <- distance_to_dataframe(mat_dist_mat)

## Computing Drug Distances based on Structural Similarity
In this step, we retrieve the SMILES representations of the drugs from the PubChem service.
We use the `fetch_smiles()` function to retrieve the SMILES representations of the drugs.
The function takes the drug names (obtained from the row names of the distance matrix `mat_dist_mat`) as input and queries the PubChem service to fetch the corresponding SMILES strings.
Next, we utilize the `chemicals_distance()` function to compute the drug distances.
The function takes the drug SMILES representations stored in the drugs_smiles object as input.
We specify the distance calculation method as "fingerprint", which utilizes molecular fingerprints to quantify the structural similarity between compounds.
Other options include "mcs" distance which is based on the maximal common substructure and "levenshtein" which is based on the edit distance of the SMILES strings.
The `python_bin` parameter represents the path to the Python interpreter, which is necessary for executing the underlying chemical similarity calculation code.
The resulting `smiles` object represents the pairwise distances between drugs based on their structural similarity.

In [ ]:
# retrieve drug SMILES from pubchem service
drugs_smiles <- fetch_smiles(rownames(mat_dist_mat))
# drug distances based on structural similarity
smiles <- chemicals_distance(drugs_smiles, method="fingerprint", python=python_bin)

## Computing Drug Distances based on Target Connectivity
In this part, we calculate the pairwise distances between drugs based on the connectivity of their respective targets in the disease network.
The connectivity is determined by the average shortest path length between the targets in the network.

We utilize the `get_avg_shortest_path()` function to compute the average shortest path length between the targets of each drug in the disease network represented by the `conGraph` object.
The function takes the disease network graph (`Graph`), the drug-target associations (`drug_target_df`), and the column indices specifying the drug and target columns in the associations data frame (`drugs_col` and `targets_col`, respectively).

The resulting `avg_drug_shortest_path` object contains the average shortest path lengths between the drugs based on their corresponding targets in the disease network.
We then convert this information into a data frame using the `distance_to_dataframe()` function.

In [ ]:
# search relevant drug targets
opentargets_idx <- map_drugs_on_graph$dat.drug.molecule_name %in% tolower(names(drugs_smiles))
filter_by_L1000_drugs <- unique(map_drugs_on_graph[opentargets_idx, ])
targets <- filter_by_L1000_drugs
# drug distances based on the shortest path length connecting their respective targets in the disease network
avg_drug_shortest_path <- get_avg_shortest_path(Graph = conGraph, drug_target_df = filter_by_L1000_drugs, drugs_col = 6, targets_col = 1)
graph <- distance_to_dataframe(avg_drug_shortest_path)

# Drug Combination Analysis with a Multi-Objective Genetic Algorithm

In this step, we perform a drug combination analysis using a multi-objective genetic algorithm.
The genetic algorithm takes into account multiple drug properties, including structural similarity, perturbational differences, network distances, and target connectivity.

We initiate the genetic algorithm by calling the `genetic_algorithm()` function.
This function takes several input parameters:

    `smiles`: The drug structural distance calculated based on the drug SMILES retrieved earlier.
    `moa`: The drug perturbational distance calculated based on the drug co-expression networks.
    `graph`: The drug network distances based on the shortest path lengths connecting their respective targets in the disease network.
    `disease_network`: The disease network used to incorporate information about the disease in the drug combination analysis.
    `rank`: The ranked gene list based on the differential expression analysis of the disease.
    `targets`: The relevant drug targets obtained from the OpenTargets database.

The other parameters specified in the function control the behavior of the genetic algorithm:

    `population_size`: The number of individuals in the initial population.
    `n_offsprings`: The number of offspring produced in each generation through crossover and mutation.
    `attribute_init_prob`: The probability of initializing each attribute (drug) in the population.
    `attribute_mutation_prob`: The probability of mutating each attribute (drug) in the population.
    `crossover_prob`: The probability of performing crossover between parent individuals.
    `mutation_prob`: The probability of mutating individual attributes.
    `n_generations`: The number of generations for which the genetic algorithm will run.
    `python`: The path to the Python executable used to run the genetic algorithm.

By running the genetic algorithm, we aim to find optimal drug combinations that optimize multiple objectives, such as structural differences, perturbational differences, network distances, and target connectivity.
The algorithm explores the solution space, iteratively evolving the population of drug combinations over multiple generations to converge towards the best solutions.

In [ ]:
#  returns a named list containing the following fields:
#  drug_names: a list with the names of the drugs considered
#  hall_of_fame: the list of best drug combinations identified during the execution of the algorithm, with their scores from the fitness function
#  logbook: the average score of each fitness function for each generation
#  population: a list of binary arrays representing the individuals in the population in the last generation of the algorithm.
#
#  The precomputed input data for the genetic algorithm can be loaded with
# smiles = read.table(file.path(intermediate_path, "smiles_dist.csv"), sep=",", header=TRUE)
# moa = read.table(file.path(intermediate_path, "moa_dist.csv"), sep=",", header=TRUE)
# graph = read.table(file.path(intermediate_path, "path_dist.csv"), sep=",", header=TRUE)
# rank = read.table(file.path(intermediate_path, "graph_rank.csv"), sep=",", header=TRUE)
# targets = read.table(file.path(intermediate_path, "drug_targets.csv"), sep=",", header=TRUE)
ga = genetic_algorithm(smiles, moa, graph, disease_network, rank, targets,
                        population_size=100,
                        n_offsprings=20,
                        attribute_init_prob=0.3,
                        attribute_mutation_prob=0.1,
                        crossover_prob=0.7,
                        mutation_prob=0.3,
                        n_generations=500,
                        python=python_bin
)
#  the logbook and the hall_of_fame of the run are stored as intermediate files run001_hof.json and run001_logbook.json

The `genetic_algorithm` function returns a named list with the following fields:

  * `drug_names`: A list containing the names of the drugs considered.
  * `hall_of_fame`: The list of the best drug combinations identified throughout the algorithm's execution, accompanied by their corresponding scores from the fitness function.
  * `logbook`: A record of the average score of each fitness function for every generation.
  * `population`: A list of drug names representing the individuals in the population during the final generation of the algorithm.

In the Jupyter notebook "genetic_algorithm_results_exploration.ipynb" we provide an exemplary demonstration of how to interpret the outcomes of the genetic algorithm in the context of our specific case study.